Import libraries and setup live inline graphics

In [ ]:
%matplotlib inline
import numpy as np
import fabio
from pynx.cdi import CDI
from pylab import imshow, figure, title, draw

In [ ]:
iobs = np.fliplr(fabio.open("/home/admin/logo/logo5mu_20sec.edf").data)
support = np.flipud(fabio.open("/home/admin/logo/mask_logo5mu_20sec.edf").data)
n = len(iobs)

In [ ]:
figure(figsize=(10, 7))
imshow(support, origin='lower')
title("Support")

Create Mask specific to this dataset (from beamstop, after symmetrization of observed data)

In [ ]:
x, y = np.meshgrid(np.arange(0, n, dtype=np.float32), np.arange(0, n, dtype=np.float32))
mask = np.logical_or(iobs != 0, np.logical_or(abs(x - 256) > 30, abs(y - 256) > 30))
mask *= np.logical_or(iobs != 0, np.logical_or(abs(x - 266) > 30, abs(y) < 495))
mask *= np.logical_or(iobs != 0, np.logical_or(abs(x - 246) > 30, abs(y) > 20))
mask *= np.logical_or(iobs != 0, np.logical_or(abs(x - 10) > 30, abs(y - 240) > 5))
mask *= np.logical_or(iobs != 0, np.logical_or(abs(x - 498) > 30, abs(y - 270) > 5))
mask = np.fliplr((mask == 0))  # 0: OK, 1: masked
figure(figsize=(10, 7))
imshow(mask, origin='lower')

Run the CDI optimization

In [ ]:
%matplotlib notebook
cdi = CDI(iobs, obj=None, support=support, mask=mask, opencl_device="K80")
figure(figsize=(8, 8))
cdi.run(500, 'HIO', verbose=50, use_mask=50)
cdi.run(100, 'ER', verbose=50, use_mask=True)
cdi.ml(40, verbose=20, reg_fac=1e4)
cdi.run(100, 'HIO', verbose=50, use_mask=True)
cdi.ml(40, verbose=20, reg_fac=1e4)
cdi.run(100, 'HIO', verbose=50, use_mask=True)
cdi.ml(40, verbose=20, reg_fac=1e4)

That was a little too easy, starting from a correct support.... Create a broader support

In [ ]:
from scipy.ndimage import gaussian_filter
support = (gaussian_filter(support.astype(np.float32),5) > 0.1).astype(np.int8)

In [ ]:
%matplotlib inline
figure(figsize=(10, 7))
imshow(support, origin='lower')

And optimize again, with support update

In [ ]:
%matplotlib notebook
cdi = CDI(iobs, obj=None, support=support, mask=mask, opencl_device="K80")
figure(figsize=(10, 7))
cdi.run(500, 'HIO', verbose=50, use_mask=50, beta=0.9) # Only free masked pixels afetr cycle 50

In [ ]:
for i in range(20):
    cdi.run(100, 'HIO', verbose=50, use_mask=True, beta=0.9)
    cdi.run(50, 'ER', verbose=50, use_mask=True)
    cdi.ml(40, verbose=20, reg_fac=1e4, )
    cdi.support_update(threshold=0.1, smooth_width=0.5+2*np.exp(-i/10), force_shrink=True)